In [2]:
import geopandas as gpd
import os
import xarray as xr
from urllib.request import urlopen
import json

In [4]:
# Load the 'naturalearth_lowres' dataset that's bundled with Geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Filter the data for USA and then load US counties
usa = world[world['name'] == "United States of America"]
counties = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_20m.zip')

# Create a dictionary to store the polygons
county_polygons = {}

for _, row in counties.iterrows():
    rma_code = row["STATEFP"] + row["COUNTYFP"]
    county_polygons[rma_code] = row["geometry"]

# Print some sample data from the dictionary
for key in list(county_polygons.keys())[:5]:
    print(key, county_polygons[key])


13239 POLYGON ((-85.141831 31.839261, -85.114031 31.89336, -85.067829 31.967358, -85.0635905429847 31.991857, -85.042029 31.975658, -84.974067 31.950209, -84.907006 31.924465, -84.910338 31.776841, -84.95864 31.777854, -85.073941 31.771441, -85.1291593155832 31.7802782680766, -85.141831 31.839261))
18111 POLYGON ((-87.5266481580824 41.1660899445116, -87.393796 41.162806, -87.275882 41.218595, -87.267298 40.736582, -87.5261364904237 40.7368850814229, -87.526014 40.895582, -87.526463234368 41.0103365222041, -87.52652 41.024837, -87.5266481580824 41.1660899445116))
19053 POLYGON ((-94.014803 40.897031, -93.55654 40.898295, -93.5568966742717 40.5796594850605, -93.597352 40.579496, -93.7743442040303 40.5775304533911, -93.84093 40.576791, -94.015492 40.5740737070096, -94.014803 40.897031))
24013 POLYGON ((-77.2170237552374 39.7202172483854, -76.9993180263332 39.7200676723526, -76.991062 39.720062, -76.7870971997576 39.7210481461113, -76.804304 39.675882, -76.8931 39.457402, -76.8873 39.44050

In [4]:
mydir = os.getcwd() # would be the MAIN folder
mydir_tmp = mydir + "/datasets" 

In [9]:

# Initialize the master dictionary
master = {}

# Loop through all .nc files in the directory
for filename in os.listdir(mydir_tmp)[:1]:
    if filename.endswith(".nc"):
        # Construct the full path to the file
        filepath = os.path.join(mydir_tmp, filename)
        
        # Open the dataset
        ds = xr.open_dataset(filepath)
        
        # Extract the values of 't2m'
        t2m_values = ds['t2m'].values
        
        # Store the filename (without extension) as a key and the t2m values as its value in the master dictionary
        key_name = os.path.splitext(filename)[0]  # get filename without extension
        master[key_name] = t2m_values

print((master['dataset_0']))



[268.67017 267.76068 272.93732 274.68646 284.96744 288.86008 293.94254
 293.01898 285.6597  276.32626 269.01077 265.95612 265.70074 273.65204
 272.5619  276.72308 285.97577 290.93524 293.09485 291.26562 282.4505
 274.5877  273.53052 269.485   268.25238 265.68045 267.60376 280.39325
 283.89035 290.14127 294.45862 290.5478  292.4685  280.9663  270.27542
 269.50867 264.75958 271.80493 273.69098 276.9583  287.05835 290.45258
 294.60748 294.0995  284.04932 281.6488  270.15564 268.36053 268.51447
 271.3192  274.4049  277.45532 281.3642  292.16412 292.85962 291.55356
 289.01944 281.95953 274.24524 268.46722 266.80792 273.0689  272.00485
 277.508   283.1046  291.14496 294.67328 295.05978 290.45825 275.73737
 273.02838 268.69623 271.8927  269.19543 274.08417 279.57697 283.9804
 289.04776 292.83685 295.55273 285.61496 276.10443 273.93744 268.07617
 269.02533 266.63495 273.88617 275.44635 284.67914 290.09955 293.97693
 295.84845 288.2817  280.88837 272.6407  272.6474  267.14572 269.38837
 272.208

In [13]:
print(list(county_polygons.keys())[20])

48089


In [18]:
prop_master = {}
for county_code in list(county_polygons.keys()):
    if f"dataset_{list(county_polygons.keys()).index(county_code)}" in master.keys():
        prop_master[county_code] = master[f"dataset_{list(county_polygons.keys()).index(county_code)}"]

In [28]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [36]:
import plotly.express as px
import pandas as pd

# 1. Extract the most recent value from each array
recent_values = {key: 2*val[-1]/(val[-13] + val[-25]) for key, val in prop_master.items()}

# 2. Create a dataframe
df = pd.DataFrame(list(recent_values.items()), columns=['FIPS', 'Temperature'])
df['FIPS'] = df['FIPS'].astype(str)
print(df)
# 3. Plot the choropleth
fig = px.choropleth(
    df, 
    geojson=counties,
    locations='FIPS',
    color='Temperature',
    color_continuous_scale="Viridis",
    scope="usa",
    hover_name=df.index
)
fig.update_geos(fitbounds="locations", visible=True)
fig.write_image("fig3.png")



      FIPS  Temperature
0    13239     0.974752
1    18111     0.955577
2    19053     0.955808
3    24013     0.966432
4    29155     0.951149
..     ...          ...
465  39031     0.952938
466  48503     0.963809
467  13159     0.963250
468  13117     0.959266
469  48243     0.971753

[470 rows x 2 columns]
